In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用google-cloud-pipeline-components的AutoML表格回归管道

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_tabular.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fpipelines%2Fgoogle_cloud_pipeline_components_automl_tabular.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_tabular.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_tabular.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本笔记本展示了如何使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)中定义的组件来构建在Vertex AI Pipelines上的AutoML表格回归工作流程。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[AutoML组件](https://cloud.google.com/vertex-ai/docs/pipelines/vertex-automl-component)。了解更多关于[表格数据的回归](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Pipelines` 和 `Google Cloud Pipeline Components` 来构建一个 `AutoML` 表格回归模型。 

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Pipelines`
- `Google Cloud Pipeline Components`
- `Vertex AI AutoML`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 创建一个 KFP pipeline：
    - 创建一个 `Dataset` 资源。
    - 训练一个 AutoML 表格回归 `Model` 资源。
    - 创建一个 `Endpoint` 资源。
    - 将 `Model` 资源部署到 `Endpoint` 资源。
- 编译 KFP pipeline。
- 使用 `Vertex AI Pipelines` 执行 KFP pipeline。

了解更多关于[Google Cloud Pipeline Components](https://google-cloud-pipeline-components.readthedocs.io/en/latest/google_cloud_pipeline_components.aiplatform.html#module-google_cloud_pipeline_components.aiplatform)。

数据集

本教程使用的数据集是来自1990年人口普查的[加利福尼亚住房数据集](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)。

该数据集预测房屋的中位价格。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您预计的使用情况生成成本估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他所需的包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components

### 重新启动运行时（仅限Colab）

为了使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。请等待直到操作完成后再继续进行下一步。⚠️</b>
</div>

### 认证您的笔记本环境（仅适用于Colab）

在谷歌Colab上认证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

UUID

如果您在一个现场教程会话中，为避免用户在创建的资源中发生命名冲突，您可以为每个实例会话创建一个UUID，并将该UUID附加到您在本教程中创建的资源名称上。这样可以确保每个资源都能被唯一标识，并避免与其他用户在会话期间创建的资源发生冲突。

In [ ]:
import random
import string


# Generate a uuid of length 8
def generate_uuid():
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=8))


UUID = generate_uuid()

### 设置 Google Cloud 项目信息

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type: "string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在的情况下才能执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

服务账号

**如果您不知道您的服务账号**，请尝试使用`gcloud`命令在下面执行第二个单元格来获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账号访问权限

运行以下命令以授予您的服务账号对在上一步中创建的存储桶中的管道工件进行读写的访问权限 - 您只需要针对每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import kfp

设置以下常量用于Vertex AI管道：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/cal_housing".format(BUCKET_URI)

初始化Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 定义AutoML表格回归模型管道

接下来，您将定义管道，使用`Dataset`资源创建和部署AutoML表格回归`Model`资源。该管道利用Google Cloud组件实现资源的无缝集成和高效管理。

In [ ]:
TRAIN_FILE_NAME = "california_housing_train.csv"
! gsutil cp gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/california_housing/california_housing_train.csv {PIPELINE_ROOT}/data/

gcs_csv_path = f"{PIPELINE_ROOT}/data/{TRAIN_FILE_NAME}"


@kfp.dsl.pipeline(name="automl-tab-training-v2")
def pipeline(project: str = PROJECT_ID, region: str = LOCATION):
    from google_cloud_pipeline_components.v1.automl.training_job import \
        AutoMLTabularTrainingJobRunOp
    from google_cloud_pipeline_components.v1.dataset import \
        TabularDatasetCreateOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    dataset_create_op = TabularDatasetCreateOp(
        project=project, display_name="housing", gcs_source=gcs_csv_path
    )

    training_op = AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name="train-automl-cal_housing",
        optimization_prediction_type="regression",
        optimization_objective="minimize-rmse",
        column_transformations=[
            {"numeric": {"column_name": "longitude"}},
            {"numeric": {"column_name": "latitude"}},
            {"numeric": {"column_name": "housing_median_age"}},
            {"numeric": {"column_name": "total_rooms"}},
            {"numeric": {"column_name": "total_bedrooms"}},
            {"numeric": {"column_name": "population"}},
            {"numeric": {"column_name": "households"}},
            {"numeric": {"column_name": "median_income"}},
            {"numeric": {"column_name": "median_house_value"}},
        ],
        dataset=dataset_create_op.outputs["dataset"],
        target_column="median_house_value",
    )

    endpoint_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name="train-automl-cal_housing_endpoint",
    )

    _ = ModelDeployOp(
        model=training_op.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

编译管道。

In [ ]:
from kfp import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tabular_regression_pipeline.yaml",
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "cal_housing_" + UUID

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular_regression_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run()

! rm tabular_regression_pipeline.yaml

点击生成的链接查看在云控制台中的运行情况。

<!-- 它在运行时应该看起来像这样：

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" width="40%"/></a> -->

在用户界面中，许多流水线DAG节点在您点击它们时展开或折叠。这是DAG的部分展开视图（点击图片查看更大版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" width="40%"/></a>

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

In [ ]:
delete_bucket = True

dataset_display_name = "housing"
pipeline_display_name = "automl-tab-training-v2"
model_display_name = "train-automl-cal_housing"
endpoint_display_name = "train-automl-cal_housing_endpoint"


endpoints = aiplatform.Endpoint.list(
    filter=f"display_name={endpoint_display_name}", order_by="create_time"
)
if endpoints:
    endpoint = endpoints[0]
    endpoint.undeploy_all()
    endpoint.delete()
    print("Deleted endpoint:", endpoint)

models = aiplatform.Model.list(
    filter=f"display_name={model_display_name}", order_by="create_time"
)
if models:
    model = models[0]
    model.delete()
    print("Deleted model:", model)

datasets = aiplatform.TabularDataset.list(
    filter=f"display_name={dataset_display_name}", order_by="create_time"
)
if datasets:
    dataset = datasets[0]
    dataset.delete()
    print("Deleted dataset:", dataset)


job.delete()


if delete_bucket:
    ! gsutil rm -r $BUCKET_URI